In [10]:
import sys,os,glob,copy
sys.path.append('../')
import numpy as np
from numpy.linalg import norm
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.interpolate import LinearNDInterpolator,interp2d
import matplotlib as mpl
from matplotlib.colors import LogNorm
from IPython.display import display, Markdown
import glob
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution


#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.cm.get_cmap('RdYlBu')


pd.set_option('display.max_rows', 200)

In [11]:
atlasData = pd.read_pickle('../ATLAS_data/CutFlow_Chargino.pcl')
atlasData['label'] = 'ATLAS'

In [12]:
file = '../scanResults_Wino2.pcl'
mLLP = 1300.0
tau_ns = 10.0
recastData = pd.read_pickle(file)
recastData['label'] = 'recast'
recastData = recastData.round({'tau_ns' : 2, 'mLLP' : 2})

rData = recastData[(recastData['mLLP'] == mLLP) & (recastData['tau_ns'] == tau_ns)]
rData

,mLLP,width,tau_ns,Regime,Target Mass [GeV],SR,Luminosity (1/fb),$N_s$,$\sigma_{Ns}$,mLSP,...,S95_obs,$\mu_{obs}$,$\mu_{exp}$,$\mu_{comb}$,$E_{T}^{miss}>170$ GeV,$p_{T} > 50$ GeV,$p_{T} > 120$ GeV,$|\eta|<1.8$,"$m_{T}({track},{p}_{{T}}^{{ miss}}) > 130$ GeV",label
268,1300.0,6.582000e-17,10.0,Long Lifetime,1300.0,SR-Inclusive_High,139.0,1.090516,0.016063,1299.0,...,10.3,9.445074,3.209491,NaN,"(7.066, 0.3858)","(4.706, 0.2569)","(4.634, 0.253)","(4.348, 0.2373)","(4.348, 0.2373)",recast
269,1300.0,6.582000e-17,10.0,Long Lifetime,1300.0,SR-Inclusive_Low,139.0,0.359215,0.006014,1299.0,...,9.7,27.003297,18.094993,NaN,"(7.066, 0.3858)","(4.706, 0.2569)","(4.634, 0.253)","(4.348, 0.2373)","(4.348, 0.2373)",recast


In [13]:
allData = atlasData
columns = [c for c in atlasData.columns if c in rData]
allData = pd.concat((allData,rData[columns]))
    
allData.sort_values(['mLLP','tau_ns'],inplace=True)        

In [14]:
allData.T

,0,268,269
mLLP,1300.0,1300.0,1300.0
tau_ns,10.0,10.0,10.0
Total,"(20.8, 1.0)","(18.32, 1.0)","(18.32, 1.0)"
Event and jet Cleaning,"(20.6, 0.995)",NaN,NaN
Trigger,"(11.2, 0.539)","(10.31, 0.563)","(10.31, 0.563)"
Primary vertex,"(11.2, 0.539)",NaN,NaN
$E_{T}^{miss}>170$ GeV,"(7.9, 0.381)","(7.066, 0.3858)","(7.066, 0.3858)"
Track in PV,"(7.8, 0.378)",NaN,NaN
$p_{T} > 50$ GeV,"(6.8, 0.326)","(4.706, 0.2569)","(4.706, 0.2569)"
Track quality requirements,"(5.8, 0.279)",NaN,NaN


In [16]:
rData.T

,268,269
mLLP,1300.0,1300.0
width,0.0,0.0
tau_ns,10.0,10.0
Regime,Long Lifetime,Long Lifetime
Target Mass [GeV],1300.0,1300.0
SR,SR-Inclusive_High,SR-Inclusive_Low
Luminosity (1/fb),139.0,139.0
$N_s$,1.090516,0.359215
$\sigma_{Ns}$,0.016063,0.006014
mLSP,1299.0,1299.0


In [6]:
rData.T

,268,269
mLLP,1300.0,1300.0
width,0.0,0.0
tau_ns,10.0,10.0
Regime,Long Lifetime,Long Lifetime
Target Mass [GeV],1300.0,1300.0
SR,SR-Inclusive_High,SR-Inclusive_Low
Luminosity (1/fb),139.0,139.0
$N_s$,1.090516,0.359215
$\sigma_{Ns}$,0.016063,0.006014
mLSP,1299.0,1299.0


In [24]:
atlasData

,mLLP,tau_ns,Total,Event and jet Cleaning,Trigger,Primary vertex,$E_{T}^{miss}>170$ GeV,Track in PV,$p_{T} > 50$ GeV,Track quality requirements,Track isolation,$p_{T} > 120$ GeV,Track momentum uncertainty,$|\eta|<1.8$,"$m_{T}({track},{p}_{{T}}^{{ miss}}) > 130$ GeV",Electron veto,Hadron/tau veto,(SR-Low - no mass Window),(SR-High - no mass Window),label
0,1300.0,10,"(20.8, 1.0)","(20.6, 0.995)","(11.2, 0.539)","(11.2, 0.539)","(7.9, 0.381)","(7.8, 0.378)","(6.8, 0.326)","(5.8, 0.279)","(5.1, 0.244)","(5.0, 0.242)","(4.8, 0.231)","(4.39, 0.211)","(3.8, 0.183)","(3.8, 0.183)","(3.8, 0.183)","(0.8, 0.039)","(1.92, 0.092)",ATLAS


In [84]:
print(recastData['(SR-Low - no mass Window)'].iloc[0][1]/recastData['(Acceptance)'].iloc[0][1])
print(recastData['(SR-High - no mass Window)'].iloc[0][1]/recastData['(Acceptance)'].iloc[0][1])

0.05078162008526765
0.11454287067740407


In [85]:
0.633 / 0.635,0.3955 / 0.390, 0.115 / 0.118,0.051 / 0.057

(0.9968503937007874, 1.014102564102564, 0.9745762711864407, 0.894736842105263)

In [96]:
s95 = {'SR-Inclusive_Low': 9.6, 'SR-Inclusive_High': 7.5}
xsecATLAS = 0.00182
for sr,nmax in s95.items():
    r = recastData[recastData['SR'] == sr]
    ns = r['$N_s$'].iloc[0]
    xsec = r['Total xsec (pb)'].iloc[0]
    lumi = r['Luminosity (1/fb)'].iloc[0]
    totalEff = ns/(xsec*1e3*lumi)
    xsecUL = nmax*1e-3/(totalEff*lumi)
    print(sr,'xsec UL (pb) = %1.5f' %xsecUL)

SR-Inclusive_Low xsec UL (pb) = 0.00730
SR-Inclusive_High xsec UL (pb) = 0.00219


In [97]:
(xsecATLAS/r['Total xsec (pb)'].iloc[0])*r['$N_s$'].iloc[0]

6.228326385159144

In [89]:
(0.00219-0.0015)/0.0015

0.46

In [29]:
0.633*0.118*0.00072*1e3*139.0

7.47537552

In [26]:
0.00182/0.001478

1.2313937753721245

In [56]:
# ATLAS values:
effSR = 0.118
lumi = 139.0
accep = 0.633
xsecCut = 0.00072/0.001478
# massWindow = 0.75
massWindow = 1.0
totalEff = xsecCut*accep*effSR*massWindow
print('totalEff=',totalEff)
sigmaULpb = 7.5*1e-3/(totalEff*lumi)
print(sigmaULpb)

totalEff= 0.036386792963464136
0.0014828686492528206


In [57]:
# Recast values:
effSRtimesAccep = 0.07254
xsecCut = 0.00072/0.001478
# massWindow = 0.75
massWindow = 1.0
totalEff = xsecCut*effSRtimesAccep*massWindow
sigmaULpb = 7.5*1e-3/(totalEff*lumi)
print('totalEff=',totalEff)
print(sigmaULpb)

totalEff= 0.03533748308525034
0.0015269008945035865


In [51]:
xsecTotal = 0.001478*1e3
Ns = 5.06
totalEff2 = Ns/(lumi*xsecTotal)
print(totalEff2)

0.02462982252898628
